# Projeto 2 - Ciência dos Dados

Beatriz Muniz de Castro e Silva

Nicole Sarvasi Alves da Costa

___
## Classificador automático de sentimento


Com o intuito de destacar mensagens que designem o nome do produto, primeiro baixamos os *tweets* com o nome do filme Bacurau. Após, classificamos um por um entre "relevante" e "irrelevantes", tal classificação será explicitada mais adiante.

Para fazer isso, utilizaremos o algoritmo "Naive Bayes", criado no século XVIII. Atualmente, esse classificador probabilístico tem sido utilizado na área de *Machine Learning* principalmente para a categorização de textos, determinação de e-mails SPAM, e até para medicina diagnóstica.

Para o treinamento do cógigo, utilizamos uma base de 300 *tweets* para podermos implementar o classificador Naive-Bayes. Isso possibilitará verificarmos sua eficiência, uma vez que poderemos comparar as classificações feitas, com as previamente estabelecidas por nós.

**Classificação dos *tweets*:**

**-Relevante**: mensagens que passem uma imagem positiva sobre o filme                    
    **ex.:** "Bacurau é um filme excelente!"

**-Irrelevante**: mensagens que passem uma imagem negativa sobre o filme ou que não o caracterizem                   
        **ex.:** "O filme bacurau é péssimo" ; "Bacurau já está em cartaz"

## Preparando o ambiente no jupyter:

In [105]:
%%capture

#Instalando o tweepy
!pip install tweepy
#Instalando o pacote emoji para limpar mensagens
!pip install emoji

In [106]:
#importando os pacotes necessários
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import functools
import operator
import numpy as np
import emoji
import re

___
## Autenticando no  Twitter

* Conta: @nicknennis

In [107]:
#Dados de autenticação do twitter:

#Identificador da conta no twitter: @nickcnennis

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

FileNotFoundError: [Errno 2] No such file or directory: 'auth.pass'

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'bacurau'

#Quantidade mínima de mensagens capturadas:
n = 700
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True)

#Inicia a captura
i = 1
msgs = []

#retira os retweets
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode='extended').items():  
    if (not msg.retweeted) and ('RT' not in msg.full_text): 
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [108]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Classificamos os *tweets* de treinamento individualmente no Excel, atribuindo 0 para "irrelevante" e 1 para "relevante".

In [109]:
#leitura do arquivo excel
treino = pd.read_excel("bacurau.xlsx", "Treinamento") #tabela da parte de treinamento do excel
teste = pd.read_excel("bacurau.xlsx", "Teste") #tabela da parte de teste do excel

___
### Montando o Classificador Naive-Bayes


Para a montagem do classificador Naive-Bayes, primeiramente limpamos os dados. Ou seja, corrigimos os espaços entre as palavras e tiramos as pontuações, tendo o cuidado de manter os emojis, os quais podem ser cruciais para determinar algum tipo de sentimento.

In [110]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    
    text_split_emoji = emoji.get_emoji_regexp().split(text)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    text = ' '.join(word for word in text_split)
    
    punctuation = '[!-/.:?;@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    nova_linha = '[\n]'
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = re.sub(nova_linha, " ", text_subbed)
    return text_subbed

Abaixo, criamos uma lista com os *tweets* de teste limpos e aproveitamos para deixar todas as palavras em letra minúscula, o que nos ajudará no momento de identificar as frequências das palavras.

In [111]:
#criar uma lista com os tweets de teste limpos
teste_t = pd.Series(teste["Teste"])
teste_limpo = []
for i in teste_t:
    teste_limpo.append(cleanup(i.lower()))
# teste_limpo

In [112]:
#criar uma lista com os tweets de treinamento limpos
treino_s = pd.Series(treino["Treinamento"])
treino_limpo = []
for i in treino_s:
    treino_limpo.append(cleanup(str(i).lower()))
# treino_limpo

Outra limpeza que poderia ser feita para o melhor rendimento do nosso classificador seria a retirada de artigos, que não se mostram cruciais para a classificação de sentimento de cada *tweet*. Para isso, seria eficaz uma função a qual retirasse palavras de até duas letras, este processo serviria também para a retirada de erros possivelmente feitos no momento de escrita nos textos.

**Frequência relativa total das palavras**

In [113]:
treino_t = (" ").join(treino_limpo) #junta todos os tweets de treinamento em uma string só
lista_relativa = treino_t.split() #divide a string pra fazer uma lista com cada palavra separadamente
frequencia_absoluta = pd.Series(lista_relativa).value_counts()
frequencia_relativa = pd.Series(lista_relativa).value_counts(True)
frequencia_relativa #frequencia relativa total das palavras

bacurau            0.059806
de                 0.044272
rt                 0.021100
e                  0.021100
a                  0.019288
é                  0.017864
que                0.017735
tirei              0.015534
o                  0.015146
eu                 0.012816
você               0.011133
do                 0.009968
ver                0.009191
pra                0.008932
no                 0.008026
um                 0.007896
assistir           0.007896
filme              0.007249
não                0.007120
uma                0.006861
com                0.006731
para               0.006731
em                 0.006084
da                 0.005955
tem                0.005696
gente              0.005437
mas                0.005437
qual               0.004919
lunga              0.004531
domingas           0.004531
                     ...   
lucasnunnes97      0.000129
temática           0.000129
clássico           0.000129
😟                  0.000129
cair               0

**Frequência relativa dos *tweets* relevantes**

In [114]:
rel = treino.index[treino.Relevância == 1] #critério para selecionar só os tweets relevantes
treino_r = treino.loc[rel, "Treinamento"]  #cria nova database com os tweets relevantes


#frequência das palavras nos tweets relevantes
treino_rt = treino_r.str.cat()
treino_rt = cleanup(treino_rt.lower())
lista_relevante = treino_rt.split()
frequencia_absoluta = pd.Series(lista_relevante).value_counts()
frequencia_rel_relevantes = pd.Series(lista_relevante).value_counts(True)
frequencia_rel_relevantes #frequência relativa dos tweets relevantes

bacurau           0.059778
de                0.034291
que               0.027340
e                 0.024096
o                 0.018999
pra               0.018536
ver               0.018072
a                 0.016682
assistir          0.014365
filme             0.013438
do                0.012975
é                 0.012975
eu                0.012512
no                0.010658
não               0.009268
um                0.007414
em                0.007414
com               0.006951
mas               0.006951
cinema            0.006951
me                0.006487
mais              0.006487
tem               0.005561
bom               0.005097
vou               0.005097
sobre             0.005097
uma               0.005097
da                0.004634
q                 0.004634
vai               0.004634
                    ...   
pararem           0.000463
sido              0.000463
silveropereira    0.000463
preço             0.000463
gostou            0.000463
exibidos          0.000463
s

**Frequência relativa dos *tweets* irrelevantes**

In [115]:
irel = treino.index[treino.Relevância == 0]
treino_ir = treino.loc[irel, "Treinamento"]  #tweets irrelevantes

#frequência das palavras nos tweets irrelevantes
treino_irt = treino_ir.str.cat()
treino_irt = cleanup(treino_irt.lower())
lista_irrelevante = treino_irt.split()
frequencia_absoluta = pd.Series(lista_irrelevante).value_counts()
frequencia_rel_irrelevantes = pd.Series(lista_irrelevante).value_counts(True)
frequencia_rel_irrelevantes #frequência relativa dos tweets irrelevantes

bacurau           0.054954
de                0.051858
é                 0.021091
e                 0.020511
a                 0.020511
que               0.014899
tirei             0.014319
você              0.014319
o                 0.014125
eu                0.011997
do                0.009481
para              0.009094
um                0.008707
uma               0.007933
no                0.007546
com               0.007159
da                0.006966
domingas          0.006772
qual              0.006772
lunga             0.006579
não               0.006385
gente             0.006192
em                0.005998
tem               0.005998
personagem        0.005998
ver               0.005805
assistir          0.005805
pra               0.005611
se                0.005418
filme             0.005224
                    ...   
acionar           0.000193
júri              0.000193
f…rt              0.000193
high              0.000193
conta             0.000193
putz              0.000193
e

Determinar as frequências acima é crucial para podermos calcular as probabilidades as quais serão utilizadas no método de Naive Bayes.

**Quantidade de *tweets* relevantes e irrelevantes**

In [116]:
quantidade_relevancia = treino.Relevância.value_counts()
quantidade_relevancia #quantidade de tweets relevantes e irrelevantes

0    354
1    147
Name: Relevância, dtype: int64

**Porcentagem de *tweets* relevantes e irrelevantes**

In [117]:
quantidade_relevancia_pct = treino.Relevância.value_counts(True)*100
quantidade_relevancia_pct #porcentagem dos tweets que são relevantes e irrelevantes

0    70.658683
1    29.341317
Name: Relevância, dtype: float64

**Lista com todas as palavras que aparecem nos *tweets* da nossa tabela de treinamento**

In [140]:
palavras_totais = [] #lista com todas as palavras sem duplicatas

for palavra in lista_relativa:
    if palavra not in palavras_totais:
        palavras_totais.append(palavra)
for palavra in lista_relevante:
    if palavra not in palavras_totais:
        palavras_totais.append(palavra)
for palavra in lista_irrelevante:
    if palavra not in palavras_totais:
        palavras_totais.append(palavra)    
print(palavras_totais)
    

['nossa', 'eu', 'desviei', 'muito', 'rápido', 'de', 'um', 'spoiler', 'bacurau', 'me', 'senti', 'ninja', 'rt', 'avokdoido', 'quem', 'nao', 'gostou', 'é', 'no', 'mínimo', 'do', 'pouco', 'nazista', 'sim', 'vi', 'ontem', 'com', 'a', 'viviane_cardoso', 'o', 'filme', 'fantástico', 'vale', 'pena', 'ver', 'se', 'você', 'tiver', 'mente', 'aberta', 'e', 'qualquer', 'coisa', 'que', 'faço', 'vivi', 'sempre', 'bom', 'companheira', 'filmes', 'vida', 'lutas', 'lt', '3', 'hoje', 'finalmente', 'vejo', 'será', 'consigo', 'chegar', 'até', 'sala', 'livre', 'tá', 'difícil', 'lzanin', 'texto', 'marcelo', 'coelho', 'chama', '‘bolsonarisno', 'sinal', 'trocado’', 'lembra', 'ideia', 'alguns', 'jornalistas', 'na', 'é…', 'diretor', 'joão', 'kleber', 'mendonça', 'filho', 'n', 'vou', 'conseguir', 'qual', 'desse', 'q', 'galera', 'ta', 'doida', 'achei', 'era', 'peixe', 'isso', 'vai', 'assistir', 'sai', 'maluco', 'todo', 'dia', 'bora', 'galeres', 'aproveitando', 'pra', 'divulgar', 'melhor', '2019', 'personagem', 'tire

In [141]:
freq_irr={}
freq_rel={}

for palavra in palavras_totais:
    freq_irr[palavra]=1
    freq_rel[palavra]=1

In [142]:
for palavra in lista_relevante:
    freq_rel[palavra]+=1
freq_rel

{'nossa': 3,
 'eu': 28,
 'desviei': 1,
 'muito': 6,
 'rápido': 1,
 'de': 75,
 'um': 17,
 'spoiler': 2,
 'bacurau': 130,
 'me': 15,
 'senti': 1,
 'ninja': 1,
 'rt': 6,
 'avokdoido': 2,
 'quem': 8,
 'nao': 5,
 'gostou': 2,
 'é': 29,
 'no': 24,
 'mínimo': 3,
 'do': 29,
 'pouco': 2,
 'nazista': 2,
 'sim': 5,
 'vi': 5,
 'ontem': 6,
 'com': 16,
 'a': 37,
 'viviane_cardoso': 2,
 'o': 42,
 'filme': 30,
 'fantástico': 2,
 'vale': 2,
 'pena': 3,
 'ver': 40,
 'se': 6,
 'você': 7,
 'tiver': 2,
 'mente': 2,
 'aberta': 2,
 'e': 53,
 'qualquer': 3,
 'coisa': 8,
 'que': 60,
 'faço': 2,
 'vivi': 2,
 'sempre': 3,
 'bom': 12,
 'companheira': 2,
 'filmes': 7,
 'vida': 4,
 'lutas': 2,
 'lt': 2,
 '3': 1,
 'hoje': 8,
 'finalmente': 3,
 'vejo': 2,
 'será': 2,
 'consigo': 2,
 'chegar': 2,
 'até': 4,
 'sala': 3,
 'livre': 2,
 'tá': 9,
 'difícil': 2,
 'lzanin': 2,
 'texto': 2,
 'marcelo': 2,
 'coelho': 2,
 'chama': 2,
 '‘bolsonarisno': 2,
 'sinal': 2,
 'trocado’': 2,
 'lembra': 2,
 'ideia': 2,
 'alguns': 2,
 'jo

In [143]:
for palavra in lista_irrelevante:
    freq_irr[palavra]+=1
freq_irr

{'nossa': 5,
 'eu': 63,
 'desviei': 2,
 'muito': 17,
 'rápido': 2,
 'de': 269,
 'um': 46,
 'spoiler': 5,
 'bacurau': 285,
 'me': 21,
 'senti': 2,
 'ninja': 2,
 'rt': 17,
 'avokdoido': 4,
 'quem': 8,
 'nao': 9,
 'gostou': 5,
 'é': 110,
 'no': 40,
 'mínimo': 7,
 'do': 50,
 'pouco': 5,
 'nazista': 4,
 'sim': 5,
 'vi': 4,
 'ontem': 1,
 'com': 38,
 'a': 107,
 'viviane_cardoso': 1,
 'o': 74,
 'filme': 28,
 'fantástico': 1,
 'vale': 2,
 'pena': 3,
 'ver': 31,
 'se': 29,
 'você': 75,
 'tiver': 1,
 'mente': 1,
 'aberta': 1,
 'e': 107,
 'qualquer': 3,
 'coisa': 5,
 'que': 78,
 'faço': 1,
 'vivi': 1,
 'sempre': 14,
 'bom': 5,
 'companheira': 5,
 'filmes': 2,
 'vida': 23,
 'lutas': 1,
 'lt': 3,
 '3': 4,
 'hoje': 8,
 'finalmente': 2,
 'vejo': 2,
 'será': 1,
 'consigo': 2,
 'chegar': 1,
 'até': 7,
 'sala': 13,
 'livre': 1,
 'tá': 11,
 'difícil': 1,
 'lzanin': 2,
 'texto': 2,
 'marcelo': 3,
 'coelho': 2,
 'chama': 2,
 '‘bolsonarisno': 2,
 'sinal': 2,
 'trocado’': 2,
 'lembra': 2,
 'ideia': 2,
 'algun

## Implementação do Naive Bayes


Para isso foi utilizado o teorema de Bayes:



$$
P(relevância|palavra) = \frac{P(palavra|relevância)P(relevância)}{P(palavra)}
$$

In [144]:
prob_irr = {}
prob_rel = {}

for palavra in palavras_totais:
    prob_rel[palavra]= freq_rel[palavra]/(len(palavras_totais)+quantidade_relevancia[1])
    prob_irr[palavra]= freq_irr[palavra]/(len(palavras_totais)+quantidade_relevancia[0])
    

#probabilidade da relevância em relação às palavras do tweet


___
### Verificando a performance

Agora vamos testar o classificador com a base de Testes.

In [145]:
teste_df = pd.DataFrame()
teste_df["Tweets"] = teste_limpo
teste_df["Relevância"] = teste["Relevância"]

teste_df.head()

,Tweets,Relevância
0,alguém me convida pra assistir bacurau amanhã,1
1,kmendoncafilho as duas sessoes de bacurau hj ...,1
2,cinearteuff bacurau tem entranhas,1
3,não é possível que bacurau já esteja disponíve...,0
4,só vou ver bacurau domingo boe alguma gay pod...,0


In [149]:
chutes = [] #lista com chutes baseados nas probabilidades
for tweet in teste_limpo:
    pI = 1
    pR = 1
    palavras = tweet.split(" ")
    
    for palavra in palavras:
        if palavra in prob_rel:
            pR*= prob_rel[palavra]
        else:
            pR *= 1/(len(palavras_totais)+quantidade_relevancia[1])
        if palavra in prob_irr:
            pI*= prob_irr[palavra]
        else:
            pI *= 1/(len(palavras_totais)+quantidade_relevancia[0])
    
    Prob_Rel = quantidade_relevancia[1]*pR
    Prob_Irr = quantidade_relevancia[0]*pI
    
    if Prob_Rel>= Prob_Irr:
        chutes.append(1)
    else:
        chutes.append(0)
#     print (Prob_Irr)
        

teste_df["Chutes"]=chutes
teste_df.head()

,Tweets,Relevância,Chutes
0,alguém me convida pra assistir bacurau amanhã,1,0
1,kmendoncafilho as duas sessoes de bacurau hj ...,1,0
2,cinearteuff bacurau tem entranhas,1,0
3,não é possível que bacurau já esteja disponíve...,0,0
4,só vou ver bacurau domingo boe alguma gay pod...,0,0


In [150]:
rel_certo = teste_df.loc[(teste_df['Relevância']==1)&(teste_df['Chutes']==1), ['Relevância','Chutes']]
rel_err = teste_df.loc[(teste_df['Relevância']!=1)&(teste_df['Chutes']==1), ['Relevância','Chutes']]
irr_certo = teste_df.loc[(teste_df['Relevância']==0)&(teste_df['Chutes']==0), ['Relevância','Chutes']]
irr_err = teste_df.loc[(teste_df['Relevância']!=0)&(teste_df['Chutes']==0), ['Relevância','Chutes']]

**Fizemos um *crosstab* para comparar os resultados dados com os resultados previstos pelo treinamento**

In [161]:
tabela_sudoku = pd.crosstab(teste_df["Relevância"], teste_df["Chutes"], rownames=["Classificação"], colnames=["Treinamento"], margins=True, normalize=True)
tabela_sudoku

Treinamento,0,1,All
Classificação,,,
0,0.38,0.00,0.38
1,0.58,0.04,0.62
All,0.96,0.04,1.00


In [165]:
print("Relevantes corretos: {0}%".format((tabela_sudoku.loc[1,1])*100))
print("Irrelevantes corretos: {0}%".format((tabela_sudoku.loc[0,0])*100))

Relevantes corretos: 4.0%
Irrelevantes corretos: 38.0%


___
### Concluindo

Percebe-se que nosso classificador marcou 4% dos *tweets* como relevantes quando eles realmente eram relevantes e 38% como irrelevantes corretamente. Esses valores representam que nosso classificador ainda precisa de ajustes e melhorias para que se torne mais preciso. 


### Aperfeiçoamento:


Como o Naive Bayes não considera a correlação entre as palavras, poderiamos fazer um programa que fizesse a combinação entre elas para desenvolver um classificador ainda mais preciso. Por exemplo, poderiamos combinar as palavras "não" e "melhor", para que o computador não classifique erroneamente um *tweet* que esteja dizendo: "Bacurau não é o melhor filme que eu já assisti" como relevante, apenas porque apresenta a palavra "melhor" nele.

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) 

[Techniques for Improving the Performance
of Naive Bayes for Text Classification](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.59.2085&rep=rep1&type=pdf)
